### __Migraciones__

***

In [559]:
import pandas as pd

Carga del dataset de Migraciones.

In [560]:
Dataset_Migraciones = 'Migraciones'
Ruta = 'C:/Users/tomas/Documents/Programación/Github/Patricionog/Laboratorio de datos/Práctico 1/'
Migraciones = pd.read_csv(Ruta + Dataset_Migraciones + '.csv')

Comienzo del filtrado. Lo primero es quedarnos con las filas de "TOT" en la columna de 'Migration by Gender Code'. Las filas FEM y MAL, en conjunto y sumadas, dan TOT, por ende sería información repetida e innecesaria para los fines del objetivo del estudio.

In [561]:
Migrantes = Migraciones[Migraciones['Migration by Gender Code'] == 'TOT']

Filtramos las columnas a las que nos importan. Además, reseteamos el índice para no generar problemas.

In [562]:
Columnas = ['Country Origin Code', 'Country Dest Code', 
            '1960 [1960]', '1970 [1970]', '1980 [1980]', '1990 [1990]', '2000 [2000]']

Migrantes = Migrantes[Columnas]
Migrantes = Migrantes.reset_index(drop=True)

Cambiamos los nombres de los atributos a los definitivos.

In [563]:
Nuevos_Nombres = {'Country Origin Code': 'id_pais_origen',
                  'Country Dest Code': 'id_pais_destino',
                  '1960 [1960]': 'cantidad_en_1960',
                  '1970 [1970]': 'cantidad_en_1970',
                  '1980 [1980]': 'cantidad_en_1980',
                  '1990 [1990]': 'cantidad_en_1990',
                  '2000 [2000]': 'cantidad_en_2000'}

Migrantes = Migrantes.rename(columns=Nuevos_Nombres)

Ya que las columnas de "cantidad_en_x" deben tener enteros, si hay strings, los convertimos en NULLS.

In [564]:
Columnas_Numericas = ['cantidad_en_1960', 'cantidad_en_1970', 'cantidad_en_1980', 
                      'cantidad_en_1990', 'cantidad_en_2000']

for Columna in Columnas_Numericas:
    Migrantes[Columna] = pd.to_numeric(Migrantes[Columna], errors='coerce')

Verificamos si hay NULLS en las columnas.

In [565]:
Count_Nulls_By_Columns = Migrantes.isnull().sum()
Count_Nulls_By_Columns

id_pais_origen         0
id_pais_destino        0
cantidad_en_1960    2516
cantidad_en_1970    2516
cantidad_en_1980    2516
cantidad_en_1990    2516
cantidad_en_2000    2516
dtype: int64

Llenamos los NULLS con 0. Esto es porque en la tabla había "." para cuando no había habido migraciones en ninguna de las columnas numéricas, por lo que 0 es un relleno correcto.

In [566]:
for Columna in Columnas_Numericas:
    Migrantes[Columna] = Migrantes[Columna].fillna(0)

Por las dudas, borramos espacios entre palabras (que suponemos no los hay, pero por si los hubiere perdidos por ahí) y al principio y al final.

In [567]:
Columnas_String = ['id_pais_origen', 'id_pais_destino']

for Columna in Columnas_String:
    Migrantes[Columna] = Migrantes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Creamos cinco DataFrames, cada uno con un año específico. Los filtramos: nos quedamos solo con las filas con valores mayores a 0, es decir, con filas con migrantes. Reseteamos el índice y les cambiamos el nombre. Luego, los concatenamos en el DataFrame final.

In [568]:
DataFrames_Cantidades = []

for Columna in Columnas_Numericas:
    DataFrame = Migrantes[['id_pais_origen', 'id_pais_destino', Columna]]
    DataFrame = DataFrame[DataFrame[Columna] > 0]
    DataFrame = DataFrame.reset_index(drop=True)
    DataFrame = DataFrame.rename(columns={Columna: 'migrantes'})
    DataFrames_Cantidades.append(DataFrame)

Migrantes = pd.concat(DataFrames_Cantidades)

Guardamos el archivo.

In [569]:
# Guardar archivo.
Migrantes.to_csv('Tablas/TABLA. Migrantes.csv', index=False)

Migrantes.head(10)

,id_pais_origen,id_pais_destino,migrantes
0,AFG,DZA,41.0
1,AFG,ARG,6.0
2,AFG,AUS,42.0
3,AFG,AUT,47.0
4,AFG,BHR,62.0
5,AFG,BGD,151.0
6,AFG,BEL,16.0
7,AFG,BRA,8.0
8,AFG,BGR,168.0
9,AFG,BDI,1.0


### __Regiones__

***

Las regiones se extraen del dataset "Sedes".

In [570]:
Dataset_Sedes = 'Sedes'
Ruta = 'C:/Users/tomas/Documents/Programación/Github/Patricionog/Laboratorio de datos/Práctico 1/'
Sedes = pd.read_csv(Ruta + Dataset_Sedes + '.csv')

Nos quedamos con las columna que nos interesa: region_geografica.

In [571]:
Regiones = pd.DataFrame(Sedes['region_geografica'])

Eliminamos espacios adicionales.

In [572]:
Regiones['region_geografica'] = Regiones['region_geografica'].str.strip().str.replace(r'\s+', ' ', regex=True)

Nos quedamos con los valores únicos.

In [573]:
Regiones = Regiones.drop_duplicates()

Las ordenamos alfabéticamente y reseteamos el índice para no generar problemas.

In [574]:
Regiones = Regiones.sort_values(by='region_geografica').reset_index(drop=True)

Generamos una nueva columna: id, con números identificatorios.

In [575]:
Regiones['id'] = [x for x in range(1,len(Regiones)+1)]

Cambiamos los nombres de los atributos y ordenamos las columnas

In [576]:
Regiones = Regiones.rename(columns = {'region_geografica': 'nombre'})
Regiones = Regiones[['id', 'nombre']]

Guardamos el documento en un .csv.

In [577]:
# Guardar archivo.
Regiones.to_csv('Tablas/TABLA. Regiones.csv', index=False)

Regiones.head(10)

,id,nombre
0,1,AMÉRICA CENTRAL Y CARIBE
1,2,AMÉRICA DEL NORTE
2,3,AMÉRICA DEL SUR
3,4,ASIA
4,5,EUROPA CENTRAL Y ORIENTAL
5,6,EUROPA OCCIDENTAL
6,7,OCEANÍA
7,8,ÁFRICA DEL NORTE Y CERCANO ORIENTE
8,9,ÁFRICA SUBSAHARIANA


### __Países__

***

Usamos los códigos y nombres de la base de datos "Migraciones.csv", ya que son más exhaustivos que los de las otras tablas. El procedimiento para encontrar los valores únicos es:

1. Armar dos DataFrames: uno con los países de origen -código y nombre- y otro con los países de destino -código y nombre-.
2. Concatenarlos verticalmente, llamando las columnas "id" y "nombre". Este DataFrame se llama "Países".
3. Se eliminan las filas con valores duplicados en la columna "id". 

In [578]:
Paises_Origen = Migraciones[['Country Origin Code', 'Country Origin Name']]
Paises_Destino = Migraciones[['Country Dest Code', 'Country Dest Name']]

Renombramos las columnas para que se llamen igual.

In [579]:
Nuevos_Nombres_Origen = {'Country Origin Code': 'id',
                         'Country Origin Name': 'nombre'}

Nuevos_Nombres_Destino = {'Country Dest Code': 'id',
                          'Country Dest Name': 'nombre'}

Paises_Origen = Paises_Origen.rename(columns = Nuevos_Nombres_Origen)
Paises_Destino = Paises_Destino.rename(columns = Nuevos_Nombres_Destino)

Concatenamos verticalmente los dos DataFrames y creamos "Paises".

In [580]:
Paises = pd.concat([Paises_Origen, Paises_Destino], axis=0)

Eliminamos espacios adicionales.

In [581]:
for Columna in Paises.columns:
    Paises[Columna] = Paises[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Filtramos los valores repetidos de las columna "id". Además, reseteamos el índice para no generar problemas.

In [582]:
Paises = Paises.drop_duplicates(subset=['id']).reset_index(drop=True)

Paises

,id,nombre
0,AFG,Afghanistan
1,ALB,Albania
2,DZA,Algeria
3,ASM,American Samoa
4,AND,Andorra
...,...,...
227,WLF,Wallis and Futuna
228,PSE,West Bank and Gaza
229,YEM,"Yemen, Rep."
230,ZMB,Zambia


Eliminamos los duplicados en la columna "pais_iso_3" en el DataFrame "Sedes". Esto es para que no se generen filas de más en el join.

In [583]:
Paises_En_Sedes = Sedes.drop_duplicates(subset='pais_iso_3')

Además, eliminamos espacios adicionales.

In [584]:
for Columna in ['region_geografica', 'pais_iso_3']:
    Paises_En_Sedes[Columna] = Paises_En_Sedes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

C:\Users\tomas\AppData\Local\Temp\ipykernel_10568\898895942.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Paises_En_Sedes[Columna] = Paises_En_Sedes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)


Agregamos el 'id_region' correspondiente a cada país. Este se extrae del correspondiente al país en el dataset "Sedes". Si no se encuentra, queda NULL.

In [585]:
Paises = pd.merge(Paises, Paises_En_Sedes, left_on='id', right_on='pais_iso_3', how='left')[['id', 'nombre', 'region_geografica']]

Hacemos un LEFT_JOIN con el DataFrame "Regiones" para extraer los id de las regiones correspondientes.

In [586]:
Paises = pd.merge(Paises, Regiones, left_on='region_geografica', right_on='nombre', how='left')

Filtramos las columnas a las que necesitamos y cambiamos nombres.

In [587]:
Paises = Paises[['id_x', 'nombre_x', 'id_y']]

Paises = Paises.rename(columns={'id_x': 'id', 'nombre_x': 'nombre', 'id_y': 'id_region'})

Paises

,id,nombre,id_region
0,AFG,Afghanistan,NaN
1,ALB,Albania,NaN
2,DZA,Algeria,8.0
3,ASM,American Samoa,NaN
4,AND,Andorra,NaN
...,...,...,...
227,WLF,Wallis and Futuna,NaN
228,PSE,West Bank and Gaza,4.0
229,YEM,"Yemen, Rep.",NaN
230,ZMB,Zambia,NaN


Contamos los NULLS y los reemplazamos con "-" (esto es opcional).

NOTA: Este punto es problemático porque hay demasiados NULLS.

In [588]:
Paises['id_region'] = Paises['id_region'].fillna('-')

Guardamos el documento.

In [589]:
# Guardar archivo.
Paises.to_csv('Tablas/TABLA. Paises.csv', index=False)

Paises.head(10)

,id,nombre,id_region
0,AFG,Afghanistan,-
1,ALB,Albania,-
2,DZA,Algeria,8.0
3,ASM,American Samoa,-
4,AND,Andorra,-
5,AGO,Angola,9.0
6,AIA,Anguilla,-
7,ATG,Antigua and Barbuda,-
8,ARG,Argentina,3.0
9,ARM,Armenia,4.0


### __Sedes__

***